#### Import Python Libraries

In [564]:
import numpy as np
import pandas as pd

#### Read input Transaction file

In [565]:
import json

with open('1537277231233_Input_Transactions.txt') as json_data:
    d = json.load(json_data)

ip_trans_df = pd.DataFrame(d)

#### Read input position file

In [566]:
ip_SOD_pos = pd.read_csv('Input_StartOfDay_Positions.txt')


#### Print expected input positions file

In [567]:
print(ip_SOD_pos)

  Instrument  Account AccountType   Quantity
0        IBM      101           E     100000
1        IBM      201           I    -100000
2       MSFT      101           E    5000000
3       MSFT      201           I   -5000000
4       APPL      101           E      10000
5       APPL      201           I     -10000
6       AMZN      101           E     -10000
7       AMZN      201           I      10000
8       NFLX      101           E  100000000
9       NFLX      201           I -100000000


#### Print input transaction file

In [568]:
ip_trans_df.columns

Index(['Instrument', 'TransactionId', 'TransactionQuantity',
       'TransactionType'],
      dtype='object')

In [569]:
print(ip_trans_df)

   Instrument  TransactionId  TransactionQuantity TransactionType
0         IBM              1                 1000               B
1        APPL              2                  200               S
2        AMZN              3                 5000               S
3        MSFT              4                   50               B
4        APPL              5                  100               B
5        APPL              6                20000               S
6        AMZN              7                 5000               S
7        MSFT              8                  300               S
8        AMZN              9                  200               B
9        APPL             10                 9000               B
10       AMZN             11                 5000               S
11       AMZN             12                   50               S


#### Total Number of instruments in the input transaction file are 4

In [570]:
ip_trans_df['Instrument'].value_counts()

AMZN    5
APPL    4
MSFT    2
IBM     1
Name: Instrument, dtype: int64

#### Total Number of instruments in the input position file are 5

In [571]:
ip_SOD_pos['Instrument'].value_counts()

MSFT    2
AMZN    2
APPL    2
IBM     2
NFLX    2
Name: Instrument, dtype: int64

### Position calculation process

#### Groupby instrument and calculate net transaction quantity in input transcation table itself based on cummulative Buy/Sell Transaction

In [588]:
#### Function to calculate net transcation quantity
def netTransaction(df1):
    df1['TransactionQuantity']=df1.apply(lambda x: -x['TransactionQuantity'] if x['TransactionType']=='S' else( x['TransactionQuantity'] if x['TransactionType']=='B' else None) , axis=1)
    group_df = df1.groupby('Instrument')['TransactionQuantity'].sum()
    df_new =group_df.to_frame()
    df_new.reset_index(inplace=True)
    return df_new
    

In [589]:
#### print new dataframe
ip_trans_new_df = netTransaction(ip_trans_df)
print(ip_trans_new_df)

  Instrument  TransactionQuantity
0       AMZN                15250
1       APPL                29300
2        IBM                 1000
3       MSFT                  350


#### Merge both the dataframes to create a new dataframe

In [590]:
calc_exp_eod_pos = pd.merge(ip_SOD_pos,ip_trans_df_new, on='Instrument',how='left')

In [591]:
print(calc_exp_eod_pos)

  Instrument  Account AccountType   Quantity  TransactionQuantity
0        IBM      101           E     100000               1000.0
1        IBM      201           I    -100000               1000.0
2       MSFT      101           E    5000000               -250.0
3       MSFT      201           I   -5000000               -250.0
4       APPL      101           E      10000             -11100.0
5       APPL      201           I     -10000             -11100.0
6       AMZN      101           E     -10000             -14850.0
7       AMZN      201           I      10000             -14850.0
8       NFLX      101           E  100000000                  NaN
9       NFLX      201           I -100000000                  NaN


In [592]:
### Fill missing values with zero
calc_exp_eod_pos.fillna(value=0, inplace=True)
### Create new column called eod_Quantity and delta
calc_exp_eod_pos['eod_Quantity'] = np.nan
calc_exp_eod_pos['delta']=np.nan

In [593]:
calc_exp_eod_pos

,Instrument,Account,AccountType,Quantity,TransactionQuantity,eod_Quantity,delta
0,IBM,101,E,100000,1000.0,NaN,NaN
1,IBM,201,I,-100000,1000.0,NaN,NaN
2,MSFT,101,E,5000000,-250.0,NaN,NaN
3,MSFT,201,I,-5000000,-250.0,NaN,NaN
4,APPL,101,E,10000,-11100.0,NaN,NaN
5,APPL,201,I,-10000,-11100.0,NaN,NaN
6,AMZN,101,E,-10000,-14850.0,NaN,NaN
7,AMZN,201,I,10000,-14850.0,NaN,NaN
8,NFLX,101,E,100000000,0.0,NaN,NaN
9,NFLX,201,I,-100000000,0.0,NaN,NaN


#### Calculate positions based on account type

In [594]:
### Function to calculate EOD quantity
def createPosition(df):
    df['eod_Quantity']= df.apply(lambda x : (x['Quantity'] + x['TransactionQuantity']) if x['AccountType']=='E' else ((x['Quantity'] - x['TransactionQuantity']) if x['AccountType']=='I' else None) ,axis=1)
    return df                        

In [595]:
calc_exp_eod_pos_new = createPosition(calc_exp_eod_pos)

In [596]:
#### Print new dataframe
calc_exp_eod_pos_new

,Instrument,Account,AccountType,Quantity,TransactionQuantity,eod_Quantity,delta
0,IBM,101,E,100000,1000.0,101000.0,NaN
1,IBM,201,I,-100000,1000.0,-101000.0,NaN
2,MSFT,101,E,5000000,-250.0,4999750.0,NaN
3,MSFT,201,I,-5000000,-250.0,-4999750.0,NaN
4,APPL,101,E,10000,-11100.0,-1100.0,NaN
5,APPL,201,I,-10000,-11100.0,1100.0,NaN
6,AMZN,101,E,-10000,-14850.0,-24850.0,NaN
7,AMZN,201,I,10000,-14850.0,24850.0,NaN
8,NFLX,101,E,100000000,0.0,100000000.0,NaN
9,NFLX,201,I,-100000000,0.0,-100000000.0,NaN


#### Calculate delta i.e. Net change in positions during the day

In [597]:
calc_exp_eod_pos['delta'] = calc_exp_eod_pos['eod_Quantity']-calc_exp_eod_pos['Quantity']

#### Drop unwanted columns

In [598]:
calc_exp_eod_pos.drop(['TransactionQuantity','Quantity'],inplace=True,axis=1)

#### Rename eod_quantity as Quantity

In [599]:
calc_exp_eod_pos = calc_exp_eod_pos.rename(columns={'eod_Quantity': 'Quantity'})

In [600]:
calc_exp_eod_pos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 5 columns):
Instrument     10 non-null object
Account        10 non-null int64
AccountType    10 non-null object
Quantity       10 non-null float64
delta          10 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 480.0+ bytes


#### Convert quantity and delta to type int

In [601]:
calc_exp_eod_pos[['delta','Quantity']]=calc_exp_eod_pos[['delta','Quantity']].astype(int)

#### Reindex as per the give format of EOD positions

In [602]:
Expected_EndOfDay_Position = calc_exp_eod_pos.reindex(['Instrument','Account','AccountType','Quantity','delta'], axis=1)

#### Print final outcome

In [603]:
print(Expected_EndOfDay_Position)

  Instrument  Account AccountType   Quantity  delta
0        IBM      101           E     101000   1000
1        IBM      201           I    -101000  -1000
2       MSFT      101           E    4999750   -250
3       MSFT      201           I   -4999750    250
4       APPL      101           E      -1100 -11100
5       APPL      201           I       1100  11100
6       AMZN      101           E     -24850 -14850
7       AMZN      201           I      24850  14850
8       NFLX      101           E  100000000      0
9       NFLX      201           I -100000000      0


#### Query: At the end of the process find instruments with largest and lowest net transaction volumes for the day. 
###### (net volume is change in positions from start of day positions to end of day positions)


##### Answer:  Instrument AMZN has the largest net transaction volume for the day and instrument NFLX has the lowest net transaction volume for the day

#### UNIT TEST on function defined for dataframes

In [511]:
import pandas as pd
from nose.tools import assert_dict_equal

input = pd.DataFrame.from_dict({
    'Instrument': ['a', 'a','c','c'],
    'TransactionId': [1,2,3, 4],
    'TransactionQuantity': [20, 30,50,50],
    'TransactionType': ['B','S','B','S']
})


In [512]:
input

,Instrument,TransactionId,TransactionQuantity,TransactionType
0,a,1,20,B
1,a,2,30,S
2,c,3,50,B
3,c,4,50,S


In [513]:
expected = pd.DataFrame.from_dict({
    'Instrument': ['a', 'c'],
    'TransactionQuantity': [-10,0]
})

In [514]:

assert_dict_equal(expected.to_dict(), netTransaction(input).to_dict(), "Unit Test failed")

In [515]:
input1 = pd.DataFrame.from_dict({
    'Instrument': ['a', 'a','c','c'],
    'Quantity': [100,100,300, -300],
    'TransactionQuantity': [20, -30,50,-50],
    'AccountType': ['E','I','E','I'],
    'eod_Quantity': [0,0,0,0]
})

In [516]:
expected1 = pd.DataFrame.from_dict({
    'Instrument': ['a','a','c', 'c'],
   'Quantity': [100,100,300, -300],
    'TransactionQuantity': [20, -30,50,-50],
    'AccountType': ['E','I','E','I'],
    'eod_Quantity': [120,130,350,-250]
})

In [517]:
assert_dict_equal(expected1.to_dict(), createPosition(input1).to_dict(), "Unit Test failed")